## LeapHand 运动学功能测试 (Jupyter Notebook)

### ⚠️ 重要使用说明

**执行顺序**: 必须按照第1个 Cell → 第2个 Cell → 第3个 Cell → ... 顺序依次执行

**限制**:
1. **第1个 Cell (启动 IsaacSim) 只能执行一次**, 重复执行会导致错误
2. 若需重新开始, 必须 **重启 Kernel** 并从第1个 Cell重新执行
3. 测试过程中 **不要随意重启 Kernel**

**功能目标**:
- 获取 LeapHand 各手指末端的位置和姿态
- 测试坐标系转换 (World ↔ Body Frame)
- 设置关节角度并观察末端变化

In [ ]:
# Cell 1: 启动 IsaacSim 环境 (⚠️ 只能执行一次!)

print("正在启动 IsaacSim 环境...")

from isaaclab.app import AppLauncher
import argparse

# 方法: 使用 Namespace 对象 (兼容 AppLauncher)
args = argparse.Namespace(
    # === 可修改区域 ===
    headless=False,         # True: 无图形界面, False: 显示可视化
    device="cuda:0",        # 设备: "cuda:0", "cuda:1", "cpu" 等
    num_envs=1,             # 环境数量 (仅供本 Notebook 使用,非 AppLauncher 参数)
    # === 可修改区域结束 ===
    
    # AppLauncher 需要的其他参数 (保持默认)
    livestream=0,
    enable_cameras=False,
    verbose=False,
    experience=""
)

# 启动 IsaacSim (关键步骤,必须在其他导入前)
app_launcher = AppLauncher(args)
simulation_app = app_launcher.app

print(f"✅ IsaacSim 环境已启动!")
print(f"   - Headless 模式: {args.headless}")
print(f"   - 设备: {args.device}")
print(f"   - 环境数: {args.num_envs}")
# Cell 2: 导入所有依赖库

import torch
import numpy as np
import isaaclab.sim as sim_utils
import isaaclab.utils.math as math_utils
from isaaclab.assets import Articulation, ArticulationCfg
from isaaclab.scene import InteractiveScene, InteractiveSceneCfg
from isaaclab.sim import SimulationContext
from isaaclab.utils import configclass
from leaphand.robots.leap import LEAP_HAND_CFG

print("✅ 所有依赖导入完成!")

正在启动 IsaacSim 环境...
[INFO][AppLauncher]: Input keyword argument `livestream=0` has overridden the environment variable `LIVESTREAM=0`.
[INFO][AppLauncher]: Using device: cuda:0
[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
[INFO][AppLauncher]: Loading experience file: /home/hac/isaac/IsaacLab/apps/isaacsim_4_5/isaaclab.python.kit
[Warning] [simulation_app.simulation_app] Modules: ['omni.kit_app'] were loaded before SimulationApp was started and might not be loaded correctly.
[Warning] [simulation_app.simulation_app] Please check to make sure no extra omniverse or pxr modules are imported before the call to SimulationApp(...)
Loading user config located at: '/home/hac/isaac/.venv/lib/python3.10/site-packages/omni/data/Kit/Isaac-Sim/4.5/user.config.json'
[Info] [carb] Logging to file: /home/hac/isaac/.v

AttributeError: '_UnixSelectorEventLoop' object has no attribute '_old_agen_hooks'

2025-11-10 06:47:41 [9,036ms] [Error] [asyncio] Exception in callback <TaskStepMethWrapper object at 0x7c50bd908670>()
handle: <Handle <TaskStepMethWrapper object at 0x7c50bd908670>()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: Cannot enter into task <Task pending name='Task-5' coro=<MainWindow._dock_windows() running at /home/hac/isaac/.venv/lib/python3.10/site-packages/isaacsim/extscache/omni.kit.mainwindow-1.0.3+d02c707b/omni/kit/mainwindow/scripts/main_window.py:71>> while another task <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /home/hac/isaac/.venv/lib/python3.10/site-packages/ipykernel/kernelbase.py:519> cb=[_wrap_awaitable.<locals>.<lambda>() at /home/hac/isaac/.venv/lib/python3.10/site-packages/tornado/gen.py:854, IOLoop.add_future.<locals>.<lambda>() at /home/hac/isaac/.venv/lib/python3.10/site-packages/tornado/ioloop.py:707]> is b

2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : TLAS limit buffer size 7374781440
2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : TLAS limit : valid false, within: false
2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : TLAS limit : decrement: 167690, decrement size: 7301033856
2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : New limit 9748724 (slope: 439, intercept: 13179904)
2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : TLAS limit buffer size 4287352704
2025-11-10 06:47:41 [9,069ms] [Warning] [rtx.scenedb.plugin] SceneDbContext : TLAS limit : valid true, within: true
2025-11-10 06:47:41 [9,328ms] [Warning] [omni.kit.browser.folder.core.models.folder_browser_model] Do not load cache for Robots because url changed:
2025-11-10 06:47:41 [9,328ms] [Warning] [omni.kit.browser.folder.core.models.folder_browser_model]  - from /home/hac/isaac/isa

2025-11-10 06:47:45 [13,039ms] [Error] [asyncio] Exception in callback <TaskStepMethWrapper object at 0x7c4f51d729e0>()
handle: <Handle <TaskStepMethWrapper object at 0x7c4f51d729e0>()>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: Cannot enter into task <Task pending name='Task-70' coro=<PrimCaching._update_usd_cache_state() running at /home/hac/isaac/.venv/lib/python3.10/site-packages/isaacsim/extscache/omni.usd-1.12.4+d02c707b.lx64.r.cp310/omni/usd/_impl/utils.py:945>> while another task <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /home/hac/isaac/.venv/lib/python3.10/site-packages/ipykernel/kernelbase.py:519> cb=[_wrap_awaitable.<locals>.<lambda>() at /home/hac/isaac/.venv/lib/python3.10/site-packages/tornado/gen.py:854, IOLoop.add_future.<locals>.<lambda>() at /home/hac/isaac/.venv/lib/python3.10/site-packages/tornado/ioloop.py:707]> is bei

[13.207s] Simulation App Startup Complete
✅ IsaacSim 环境已启动!
   - Headless 模式: False
   - 设备: cuda:0
   - 环境数: 1
成功阈值: 0.2
成功阈值: 0.2
✅ 所有依赖导入完成!


: 

In [ ]:
# Cell 3: 定义场景配置并初始化

@configclass
class KinematicsTestSceneCfg(InteractiveSceneCfg):
    """运动学测试场景配置"""
    
    # 地面
    ground = sim_utils.GroundPlaneCfg()
    
    # LeapHand 机器人
    robot: ArticulationCfg = LEAP_HAND_CFG.replace(prim_path="{ENV_REGEX_NS}/Robot")

# 创建仿真上下文
sim_cfg = sim_utils.SimulationCfg(dt=0.01, device=args.device)
sim = SimulationContext(sim_cfg)

# 设置主相机视角
sim.set_camera_view(eye=[0.3, 0.3, 0.3], target=[0.0, 0.0, 0.0])

# 创建场景
scene_cfg = KinematicsTestSceneCfg(num_envs=args.num_envs, env_spacing=1.0)
scene = InteractiveScene(scene_cfg)

print(f"✅ 场景创建完成! 环境数量: {scene.num_envs}")
print(f"机器人关节数: {scene['robot'].num_joints}")
print(f"机器人 Body 数量: {scene['robot'].num_bodies}")
print(f"Body 名称: {scene['robot'].body_names}")

In [ ]:
# Cell 4: 初始化仿真并重置场景

# 重置场景
scene.reset()

# 运行几步仿真使其稳定
print("正在初始化物理仿真...")
for _ in range(10):
    scene.write_data_to_sim()
    sim.step()
    scene.update(sim_cfg.dt)

print("✅ 仿真初始化完成!")

# 获取机器人引用 (方便后续使用)
robot = scene["robot"]

---

## 运动学 API 测试区域

从此处开始可以自由交互式测试,以下 Cell 可多次执行

In [ ]:
# Cell 5: 查看所有 Body 的位置和姿态 (World 坐标系)

# 获取所有 body 的位置和四元数姿态
body_pos_w = robot.data.body_pos_w[0]  # shape: (num_bodies, 3)
body_quat_w = robot.data.body_quat_w[0]  # shape: (num_bodies, 4), wxyz 格式

print("=" * 60)
print("LeapHand 各 Body 状态 (World 坐标系)")
print("=" * 60)

for i, body_name in enumerate(robot.body_names):
    pos = body_pos_w[i]
    quat = body_quat_w[i]
    print(f"\n{i:2d}. {body_name:20s}")
    print(f"    位置:    [{pos[0]:7.4f}, {pos[1]:7.4f}, {pos[2]:7.4f}]")
    print(f"    姿态(q): [{quat[0]:7.4f}, {quat[1]:7.4f}, {quat[2]:7.4f}, {quat[3]:7.4f}]")

In [ ]:
# Cell 6: 获取指尖 (fingertip) 的位置 - 示例

# LeapHand 的指尖 body 名称 (根据 leap.py 中的注释)
fingertip_names = {
    "食指": "index_tip_head",
    "拇指": "thumb_tip_head", 
    "中指": "middle_tip_head",
    "无名指": "ring_tip_head"
}

print("=" * 60)
print("LeapHand 指尖位置")
print("=" * 60)

for finger_name, body_name in fingertip_names.items():
    try:
        body_idx = robot.body_names.index(body_name)
        pos = robot.data.body_pos_w[0, body_idx]
        quat = robot.data.body_quat_w[0, body_idx]
        
        print(f"\n{finger_name} ({body_name}):")
        print(f"  位置: [{pos[0]:.4f}, {pos[1]:.4f}, {pos[2]:.4f}]")
        print(f"  姿态: [{quat[0]:.4f}, {quat[1]:.4f}, {quat[2]:.4f}, {quat[3]:.4f}]")
    except ValueError:
        print(f"\n⚠️ {finger_name}: Body '{body_name}' 不存在")

In [ ]:
# Cell 7: 坐标系转换示例 - World → Body Frame

# 获取 root (手掌) 的姿态
root_pos_w = robot.data.root_pos_w[0]  # 手掌位置 (world)
root_quat_w = robot.data.root_quat_w[0]  # 手掌姿态四元数 (world)

print("=" * 60)
print("坐标系转换测试: World → Body Frame")
print("=" * 60)

# 示例: 将食指指尖位置从 World 坐标系转换到 Body (手掌) 坐标系
try:
    fingertip_idx = robot.body_names.index("index_tip_head")
    fingertip_pos_w = robot.data.body_pos_w[0, fingertip_idx]
    fingertip_quat_w = robot.data.body_quat_w[0, fingertip_idx]
    
    # 方法1: 使用 subtract_frame_transforms
    fingertip_pos_b, fingertip_quat_b = math_utils.subtract_frame_transforms(
        root_pos_w, root_quat_w,
        fingertip_pos_w, fingertip_quat_w
    )
    
    print(f"\n手掌 (Root) - World 坐标系:")
    print(f"  位置: {root_pos_w}")
    print(f"  姿态: {root_quat_w}")
    
    print(f"\n食指指尖 - World 坐标系:")
    print(f"  位置: {fingertip_pos_w}")
    
    print(f"\n食指指尖 - Body (手掌) 坐标系:")
    print(f"  位置: {fingertip_pos_b}")
    print(f"  姿态: {fingertip_quat_b}")
    
except ValueError as e:
    print(f"⚠️ 错误: {e}")

In [ ]:
# Cell 8: 关节角度设置与观察

# 查看当前关节位置
print("=" * 60)
print("关节状态")
print("=" * 60)

joint_pos = robot.data.joint_pos[0]  # shape: (num_joints,)
joint_vel = robot.data.joint_vel[0]

print(f"\n关节名称: {robot.joint_names}")
print(f"\n当前关节位置: {joint_pos}")
print(f"当前关节速度: {joint_vel}")

In [ ]:
# Cell 9: 设置关节目标位置并仿真 (示例: 让食指弯曲)

# 创建目标关节位置 (全部设为 0,然后修改特定关节)
target_joint_pos = torch.zeros_like(robot.data.joint_pos)

# 设置食指相关关节 (假设前3个关节是食指)
# 注意: 需要根据实际的 joint_names 调整索引
target_joint_pos[0, :3] = torch.tensor([0.5, 0.5, 0.5])  # 弯曲食指

print(f"目标关节位置: {target_joint_pos[0]}")

# 应用关节目标
robot.set_joint_position_target(target_joint_pos)

# 仿真几步观察效果
print("\n正在仿真...")
for i in range(100):
    robot.write_data_to_sim()
    sim.step()
    robot.update(sim_cfg.dt)

print("✅ 仿真完成! 请在可视化界面观察机器人状态")

# 打印新的指尖位置
fingertip_idx = robot.body_names.index("index_tip_head")
new_pos = robot.data.body_pos_w[0, fingertip_idx]
print(f"\n食指指尖新位置: {new_pos}")

In [ ]:
# Cell 10: 重置机器人到初始状态

# 重置环境
scene.reset()
print("✅ 机器人已重置到初始状态")

# 运行几步稳定仿真
for _ in range(10):
    scene.write_data_to_sim()
    sim.step()
    scene.update(sim_cfg.dt)

---

## 清理与关闭

**⚠️ 执行以下 Cell 后需重启 Kernel 才能重新开始**

In [ ]:
# Cell 11: 关闭仿真 (可选,通常不需要在 Notebook 中执行)

# simulation_app.close()
print("如需关闭仿真,请重启 Kernel")